#Analysis of Ad-Hoc Communications Network
The user can supply the overall size of desired coverage footprint andthen we will determine the following:

- Given an overall desired coverage footprint and a sequence of n communications
towers, what is the resulting resolved coverage?
- What is the total area of coverage relative to the desired total coverage area of the
original footprint? That is, are there any gaps in coverage?
- On average, how many communications towers are required before full coverage is
obtained?

Inputs w and h represent width and height of the desired region map to analyze, respectively 

In [2]:
%matplotlib inline

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def create_map(w,h):
    """
    Create a blank region area based on user input w and h, and plot the empty map
    :param w: width of the desired area 
    :param h: height of desired area
    :return: numpy area of 0's which will represent the blank map area
    """
    assert isinstance(w, int)
    assert isinstance(h, int)
    assert w>0
    assert h>0
    
    xvalues = np.array(range(w));
    yvalues = np.array(range(h));
    
    xx, yy = np.meshgrid(xvalues, yvalues)
    
    blank = np.zeros(h, w)
    plt.plot(xx, yy, marker='.', color='k', linestyle='none')
    return blank

Input n represents the numbers of towers

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches


def add_tower(n,w,h, data):
    """
    Adds tower, and returns so that each step can be shown in a visualization
    :param n: number of towers remaining
    :param w: width of region map
    :param h: height of region map
    :param data: the current map with current amount of towers
    :return: updated map with one tower added
    """
    import random
    import itertools
    assert isinstance(n, int)
    assert isinstance(w,int)
    assert isinstance(h,int)
    assert n>0 and w>0 and h>0
    
    # determines tower random width and height of rectangle (minimum 1)
    tow_width = random.randint(2, w+1)
    tow_height = random.randint(2, h+1)

    # determines which corner to start at, such that the tower is completely within the map
    s_cornerw = random.randint(0, (w+1)-tow_width)
    s_cornerh = random.randint(0, (h+1)-tow_height)

    # update numpy array if there are still towers left to be placed
    # tow_placed is the count of how many towers we've placed so far
    global tow_placed
    global rectangle_data 
    tow_placed = tow_placed + 1

    data1 = np.copy(data)
    
    # add the new tower in, with overlapping areas and no trimming yet
    data[s_cornerh:(s_cornerh+tow_height), s_cornerw:(s_cornerw+tow_width)] += tow_placed
    
    # call max_rectangle to get the new tower coverage area
    rectangle = max_rectangle(data, tow_placed)
    
    #data[data > tow_placed] = 1
    #data[data == tow_placed] = 0
    
    if rectangle == 'Area already covered':
        while(rectangle=='Area already covered'):
            rectangle = max_rectangle(data, tow_placed)
            
    elif rectangle != 'Area already covered':
        rectangle_data.append([rectangle])
        s_cornerw1 = rectangle[0][1]
        s_cornerh1 = rectangle[0][0] - rectangle[2] + 1
        tow_height1 = rectangle[2]
        tow_width1 = rectangle[1]
        data1[s_cornerh1:(s_cornerh1+tow_height1), s_cornerw1:(s_cornerw1+tow_width1)] += tow_placed
    
    return data1

In [11]:
def max_rectangle(data, tow_placed):
    """
    Updates the numpy array with the new rectangle 
    :param data: non trimmed map, numpy array
    :param tow_placed: number of the tower being placed, int
    :return: 
    """
    
    R = len(data) 
    C = len(data[0])
    
    # A will be transformed to represent a "histogram" of the coverage area
    A = np.copy(data)

    # copies the map to A. overlap areas are greater than tow_placed, 
    # array values that are == to tow_placed represent new coverage areas which will be
    #  represented by "1"; remove all values but the new coverage areas and make these "0"
    fully_enclosed = 0
    for each_row in range(0, R):
        for each_column in range(0, C):
            if A[each_row][each_column] == tow_placed:
                A[each_row][each_column] = 1
            else:
                A[each_row][each_column] = 0
                
    # if area is already fully covered, return an error string
    if np.count_nonzero(A) == 0:
        return "Area already covered"
    # turns A into a "histogram" of coverage per column for tower number tow_placed
    # The idea is to update each column of a given row with corresponding column of previous row
    for each_row in range(1, R):
        for each_column in range(0, C):
            if A[each_row][each_column] == 1:
                A[each_row][each_column] += A[each_row-1][each_column]
                
    # call maxHistArea, and find which rectangle under the "histogram" has the most area.
    # make the area of the first row the current max_area
    max_area = sum(A[0])
    maxarea_row = 0
    for each_row in range (0,R):
        if maxHistArea(A[each_row]) > max_area:
            max_area = maxHistArea(A[each_row])
            maxarea_row = each_row

            # maxarea_row stores the row number where histogram row max_area was found in. 
            # this row number is where the the bottom most row in our max rectangle as well
    
    # Using the maxarea_row number from our "histogram" matrix, and the max area value, we can find the length, width, 
    # and coordinates of the resulting rectangle by using the structure of a histogram

    for i in A[maxarea_row]:
        if (sum(check(A[maxarea_row], i)) * i) == max_area:
            rec_height = i+1
            rec_width = sum(check(A[maxarea_row], i))+1
    rec_height = 1
    rec_width = 1
    # coordinates of starting corner of rectangle      
    start_corner = np.where((A[maxarea_row]) >= rec_height)
    start_corner_col = start_corner[0][0]
    start_corner_row = maxarea_row - (rec_width+1)
    
    return [[start_corner_row, start_corner_col], rec_width, int(rec_height)]

In [1]:
def maxHistArea(heights):
    """"
    Given a list of "histogram" heights we will find the maximum possible rectangle area
    :param heights: list of "histogram" heights
    :return int: area of the biggest rectangle in that "histogram"
    """
    increasing, area, i = [], 0, 0
    while i <= len(heights):
        if not increasing or (i < len(heights) and heights[i] > heights[increasing[-1]]):
            increasing.append(i)
            i += 1
        else:
            last = increasing.pop()
            if not increasing:
                area = max(area, heights[last] * i)
            else:
                area = max(area, heights[last] * (i - increasing[-1] - 1 ))
    return area


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines 
import matplotlib.patches as patches


def plot_tower(rectangle_data):
    '''
    :param n: 
    :param s_cornerw: corner x coordinate 
    :param s_cornerh: corner y coordinate
    :param tow_width: rectangle width
    :param tow_height: rectangle height
    :return: 
    '''
    
    # better way to plot 
    # patches.Rectangle((s_cornerw, s_cornerh), tow_width-1, tow_height-1)
    
    figure = plt.figure()
    ax = figure.add_subplot(111, aspect='equal')
    for i in rectangle_data:
        current_rec = patches.Rectangle(((i[0][0] - i[2] + 1), i[0][1]), i[1] - 1, i[2] - 1, color='g') 
        ax.add_patch(current_rec)
        yield figure

In [14]:
def check(list1, val):
    """
    Given a list, create a list where entries are 1 if value is greater than or equal to val, and 0 otherwise
    :param list1: list entered 
    :param val: val to compare to
    :return: binary list
    """
    # traverse in the list
    list2 = []
    for x in list1:
 
        # compare with all the values
        # with val
        if x >= val:
            list2.append(1)
        else:
            list2.append(0)
            
    return list2

In [16]:
def main(n, w, h):
    """
    :param n: number of towers to be placed, is decremented every time a tower is placed
    :param w: width of map
    :param h: height of map
    :return: 
    """
    # every rectangle stored in the form [[corner_w, corner_h], width, height]
    rectangle_data = []
    tow_placed = 0
    region_map = create_map(w, h)
    
    for n_count in xrange(n):
        region_map = add_tower(n_count, w, h, region_map)
    
    gen = plot_tower(rectangle_data)
    # iterate through generator 
    for n_count in gen:
        next(gen)
        
